In [1]:
import pandas as pd
import inflection
pd.DataFrame.iteritems = pd.DataFrame.items

In [2]:
df = pd.read_csv('dataset/zomato.csv')

In [3]:
# LIMPEZA E FUNCOES DE TRATAMENTO DE DADOS

# ----------------------------------------
# Renomear colunas do DataFrame
# ----------------------------------------
def rename_columns(df):
    df1 = df.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df1.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df1.columns = cols_new
    return df1

df1 = rename_columns(df)

# ----------------------------------------
# Preenchendo o nome dos paises
# ----------------------------------------
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

def country_name(country_id):
    return COUNTRIES[country_id]

df1['country_code'] = df1['country_code'].apply(country_name)


# -------------------------------------------------------------------
# Categorizar todos os restaurantes somente por um tipo de culinária
# -------------------------------------------------------------------
df1["cuisines"] = df1.loc[:, "cuisines"].astype(str).apply(lambda x: x.split(",")[0])


# --------------------------------------
# Criar tipo de categoria de comida
# --------------------------------------
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
    
df1['price_range'] = df1['price_range'].apply(create_price_tye)


# --------------------------------------
# Criar nome das cores
# --------------------------------------
COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]

df1['rating_color'] = df1['rating_color'].apply(color_name)

# --------------------------------------
# Remover duplicatas
# --------------------------------------
df1 = df1.drop_duplicates()

# --------------------------------------
# Apagando linhas com valores nulos das colunas (restaurant_id, restaurant_name, country_code, city, address, locality, locality_verbose, longitude,
# latitude, cuisines, average_cost_for_two, currency, price_range, aggregate_rating, rating_color, rating_text, votes)
# --------------------------------------
df1 = (df1.dropna(subset=['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address', 'locality', 'locality_verbose', 
                          'longitude', 'latitude', 'cuisines', 'average_cost_for_two', 'currency', 'price_range', 'aggregate_rating', 
                          'rating_color', 'rating_text', 'votes']))




In [4]:
df1.head(3)


,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,currency,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,expensive,4.6,darkgreen,Excellent,619
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,Botswana Pula(P),0,0,0,0,gourmet,4.7,darkgreen,Excellent,469
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,Botswana Pula(P),0,0,0,0,expensive,4.4,green,Very Good,867


# Geral

1. Quantos restaurantes únicos estão registrados?


In [5]:
restaurantes_unicos = df1['restaurant_id'].nunique()
print(restaurantes_unicos)

6942


2. Quantos países únicos estão registrados?


In [6]:
paises_unicos = df1['country_code'].nunique()
print(paises_unicos)

15


3. Quantas cidades únicas estão registradas?


In [7]:
cidades_unicas = df1['city'].nunique()
print(cidades_unicas)

125


4. Qual o total de avaliações feitas?


In [8]:
total_avaliacoes = df1['votes'].sum()
print(total_avaliacoes)

4195634


5. Qual o total de tipos de culinária registrados?


In [9]:
total_tipos_culinaria = df1['cuisines'].nunique()
print(total_tipos_culinaria)

166


# Pais

1. Qual o nome do país que possui mais cidades registradas?


In [10]:
pais_mais_cidades = (df1.loc[:, ['country_code', 'city']]
                     .groupby('country_code')
                     .nunique()
                     .sort_values(by='city', ascending=False).head(1))
print(pais_mais_cidades)

              city
country_code      
India           49


2. Qual o nome do país que possui mais restaurantes registrados?


In [11]:
pais_mais_restaurantes = (df1.loc[:, ['country_code', 'restaurant_id']]
                          .groupby('country_code')
                          .nunique()
                          .sort_values(by='restaurant_id', ascending=False).head(1))
print(pais_mais_restaurantes)

              restaurant_id
country_code               
India                  3120


3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4
registrados?


In [12]:
filtro = df1['price_range'] == 4

pais_restaurantes_nivel_4 = (df1.loc[filtro, ['country_code', 'restaurant_id']]
                             .groupby('country_code')
                             .nunique()
                             .sort_values(by='restaurant_id', ascending=False).head(1))
print(pais_restaurantes_nivel_4)

Empty DataFrame
Columns: [restaurant_id]
Index: []


4. Qual o nome do país que possui a maior quantidade de tipos de culinária
distintos?

In [13]:
df1["cuisines"] = df1.loc[:, "cuisines"].astype(str).apply(lambda x: x.split(",")[0])

pais_mais_tipos_culinaria = (df1.loc[:, ['country_code', 'cuisines']]
                             .groupby('country_code')
                             .nunique()
                             .sort_values(by='cuisines', ascending=False).head(1))
print(pais_mais_tipos_culinaria)

              cuisines
country_code          
India               78


5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [14]:
pais_mais_avaliacoes = df1.loc[:, ['country_code', 'votes']].groupby('country_code').sum().sort_values(by='votes', ascending=False).head(1)
print(pais_mais_avaliacoes)

                votes
country_code         
India         2800199


6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem
entrega?


In [15]:
pais_mais_entregas = (df1.loc[:, ['country_code', 'is_delivering_now']]
                      .groupby('country_code')
                      .sum()
                      .sort_values(by='is_delivering_now', ascending=False).head(1))
print(pais_mais_entregas)

              is_delivering_now
country_code                   
India                      1150


7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam
reservas?

In [16]:
pais_mais_reservas = (df1.loc[:, ['country_code', 'has_table_booking']]
                      .groupby('country_code')
                      .sum()
                      .sort_values(by='has_table_booking', ascending=False).head(1))
print(pais_mais_reservas)

              has_table_booking
country_code                   
India                       256


8. Qual o nome do país que possui, na média, a maior quantidade de avaliações
registrada?

In [17]:
pais_media_mais_avaliacoes = (df1.loc[:, ['country_code', 'votes']]
                              .groupby('country_code')
                              .mean()
                              .sort_values(by='votes', ascending=False).head(1))
print(pais_media_mais_avaliacoes)

                 votes
country_code          
Indonesia     1112.825


9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [18]:
pais_maior_nota_media = (df1.loc[:, ['country_code', 'aggregate_rating']]
                         .groupby('country_code')
                         .mean()
                         .sort_values(by='aggregate_rating', ascending=False).head(1))
print(pais_maior_nota_media)

              aggregate_rating
country_code                  
Indonesia              4.60125


10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [19]:
pais_menor_nota_media = (df1.loc[:, ['country_code', 'aggregate_rating']]
                         .groupby('country_code')
                         .mean()
                         .sort_values(by='aggregate_rating', ascending=True).head(1))
print(pais_menor_nota_media)

              aggregate_rating
country_code                  
Brazil                3.321667


11. Qual a média de preço de um prato para dois por país?


In [20]:
df1["average_cost_for_two"] = df1.loc[:, "average_cost_for_two"].astype(int)

media_preco_prato_pais = (df1.loc[:, ['country_code', 'average_cost_for_two']]
                          .groupby('country_code')
                          .mean())
print(media_preco_prato_pais)

                          average_cost_for_two
country_code                                  
Australia                        138959.783333
Brazil                              138.812500
Canada                               41.861111
England                              43.510000
India                               703.602564
Indonesia                        303000.000000
New Zeland                           62.154812
Philippines                        1227.825000
Qatar                               174.000000
Singapure                           141.437500
South Africa                        339.228324
Sri Lanka                          2579.375000
Turkey                              128.584906
United Arab Emirates                153.716667
United States of America             55.018868


# Cidade

1. Qual o nome da cidade que possui mais restaurantes registrados?


In [21]:
cidade_mais_restaurantes = (df1.loc[:, ['city', 'restaurant_id']]
                            .groupby('city')
                            .nunique()
                            .sort_values(by='restaurant_id', ascending=False).head(1))
print(cidade_mais_restaurantes)

           restaurant_id
city                    
Abu Dhabi             80


2. Qual o nome da cidade que possui mais restaurantes com nota média acima de
4?

In [22]:
# Cidade com mais restaurantes com nota media acima de 4

filtro = df1['aggregate_rating'] > 4

cidade_restaurantes_nota_media_acima_4 = (df1.loc[filtro, ['city', 'restaurant_id']]
                               .groupby('city')
                               .nunique()
                               .sort_values(by='restaurant_id', ascending=False).head(1))
print(cidade_restaurantes_nota_media_acima_4)

           restaurant_id
city                    
Bangalore             79


3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de
2.5?

In [23]:
filtro = df1['aggregate_rating'] < 2.5

cidade_restautantes_nota_media_abaixo_2_5 = (df1.loc[filtro, ['city', 'restaurant_id']]
                                            .groupby('city')
                                            .nunique()
                                            .sort_values(by='restaurant_id', ascending=False).head(1))
print(cidade_restautantes_nota_media_abaixo_2_5)

         restaurant_id
city                  
Gangtok             40


4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [24]:
cidade_maior_valor_medio_prato_dois = (df1.loc[:, ['city', 'average_cost_for_two']]
                                      .groupby('city')
                                      .mean()
                                      .sort_values(by='average_cost_for_two', ascending=False).head(1))
print(cidade_maior_valor_medio_prato_dois)

          average_cost_for_two
city                          
Adelaide         416734.133333


5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária
distintas?

In [25]:
cidade_mais_tipos_culinaria = (df1.loc[:, ['city', 'cuisines']]
                               .groupby('city')
                               .nunique()
                               .sort_values(by='cuisines', ascending=False).head(1))
print(cidade_mais_tipos_culinaria)

            cuisines
city                
Birmingham        32


6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem
reservas?


In [26]:
cidade_mais_reservas = (df1.loc[:, ['city', 'has_table_booking']]
                        .groupby('city')
                        .sum()
                        .sort_values(by='has_table_booking', ascending=False).head(1))
print(cidade_mais_reservas)

           has_table_booking
city                        
Bangalore                 42


7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem
entregas?

In [27]:
cidade_mais_entregas = (df1.loc[:, ['city', 'is_delivering_now']]
                        .groupby('city')
                        .sum()
                        .sort_values(by='is_delivering_now', ascending=False).head(1))
print(cidade_mais_entregas)

          is_delivering_now
city                       
Amritsar                 48


8. Qual o nome da cidade que possui a maior quantidade de restaurantes que
aceitam pedidos online?

In [28]:
cidade_mais_pedidos_online = (df1.loc[:, ['city', 'has_online_delivery']]
                              .groupby('city')
                              .sum()
                              .sort_values(by='has_online_delivery', ascending=False).head(1))
print(cidade_mais_pedidos_online)

        has_online_delivery
city                       
Bhopal                   75


# Restaurantes

1. Qual o nome do restaurante que possui a maior quantidade de avaliações?


In [29]:
restaurante_mais_votos = df1.loc[:, ['restaurant_name', 'votes']].sort_values(by='votes', ascending=False).head(1)
print(restaurante_mais_votos)

     restaurant_name  votes
3928        Bawarchi  41333


2. Qual o nome do restaurante com a maior nota média?


In [30]:
restaurante_maior_nota_media = df1.loc[:, ['restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False).head(1)
print(restaurante_maior_nota_media)

               restaurant_name  aggregate_rating
1535  Di Rienzo Grocery & Deli               4.9


3. Qual o nome do restaurante que possui o maior valor de uma prato para duas
pessoas?


In [31]:
restaurante_maior_valor_prato_2 = df1.loc[:, ['restaurant_name', 'average_cost_for_two']].sort_values(by='average_cost_for_two', ascending=False).head(1)
print(restaurante_maior_valor_prato_2)

                  restaurant_name  average_cost_for_two
385  d'Arry's Verandah Restaurant              25000017


4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor
média de avaliação?

In [32]:
filtro = df1['cuisines'] == 'Brazilian'

culinaria_brasileira_menor_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=True).head(1))
print(culinaria_brasileira_menor_media_avaliacao)

                restaurant_name  aggregate_rating
166  NAVE MÃE FAST FOOD GOURMET               0.0


5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que
possui a maior média de avaliação?

In [33]:
# restaurante de culinária brasileira do brasil com maior media de avaliacao
filtro = (df1['country_code'] == 'Brazil') & (df1['cuisines'] == 'Brazilian')

culinaria_brasileira_maior_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                                .sort_values(by='aggregate_rating', ascending=False).head(1))
print(culinaria_brasileira_maior_media_avaliacao)

       restaurant_name  aggregate_rating
207  Braseiro da Gávea               4.9


6. Os restaurantes que aceitam pedido online são também, na média, os
restaurantes que mais possuem avaliações registradas?

In [34]:
# Testando e comparando se os restaurantes que aceitam pedidos online e os que nao aceitam possuem a mesma media de avaliacoes
filtro = df1['has_online_delivery'] == 1

restaurantes_online = df1.loc[filtro, ['restaurant_name', 'votes']].groupby('restaurant_name').mean().sort_values(by='votes', ascending=False).head(1)
print('Aceitam pedido online')
print(restaurantes_online)
print('=====================')
filtro = df1['has_online_delivery'] == 0

restaurantes_nao_online = df1.loc[filtro, ['restaurant_name', 'votes']].groupby('restaurant_name').mean().sort_values(by='votes', ascending=False).head(1)
print('Nao aceitam pedido online')
print(restaurantes_nao_online)

Aceitam pedido online
                        votes
restaurant_name              
Bawarchi         13903.666667
Nao aceitam pedido online
                   votes
restaurant_name         
Toit             15270.0


7. Os restaurantes que fazem reservas são também, na média, os restaurantes que
possuem o maior valor médio de um prato para duas pessoas?

In [35]:
filtro = df1['has_table_booking'] == 1

restaurantes_online = df1.loc[filtro, ['restaurant_name', 'average_cost_for_two']].groupby('restaurant_name').mean().sort_values(by='average_cost_for_two', ascending=False).head(1)
print('Fazem reservas')
print(restaurantes_online)
print('=====================')
filtro = df1['has_table_booking'] == 0

restaurantes_nao_online = df1.loc[filtro, ['restaurant_name', 'average_cost_for_two']].groupby('restaurant_name').mean().sort_values(by='average_cost_for_two', ascending=False).head(1)
print('Nao fazem reservas')
print(restaurantes_nao_online)

Fazem reservas
                              average_cost_for_two
restaurant_name                                   
d'Arry's Verandah Restaurant            25000017.0
Nao fazem reservas
                        average_cost_for_two
restaurant_name                             
The Café - Hotel Mulia             1200000.0


8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América
possuem um valor médio de prato para duas pessoas maior que as churrascarias
americanas (BBQ)?


In [36]:
filtro = (df1['country_code'] == 'United States of America') & (df1['cuisines'] == 'Japanese')

japoneses_eua = (df1.loc[filtro, ['restaurant_name', 'average_cost_for_two']]
                 .groupby('restaurant_name')
                 .mean()
                 .sort_values(by='average_cost_for_two', ascending=False).head(1))

filtro = (df1['country_code'] == 'United States of America') & (df1['cuisines'] == 'BBQ')

churrascarias_eua = (df1.loc[filtro, ['restaurant_name', 'average_cost_for_two']]
                     .groupby('restaurant_name')
                     .mean()
                     .sort_values(by='average_cost_for_two', ascending=False).head(1))

print(japoneses_eua)
print(churrascarias_eua)

                 average_cost_for_two
restaurant_name                      
Nobu                            250.0
                            average_cost_for_two
restaurant_name                                 
Lamberts Downtown Barbecue                 100.0


# Tipos de Culinária


1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do
restaurante com a maior média de avaliação?


In [37]:
filtro = df1['cuisines'] == 'Italian'

restaurante_italiano_maior_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=False).head(1))
print(restaurante_italiano_maior_media_avaliacao)

                  restaurant_name  aggregate_rating
366  Andre's Cucina & Polenta Bar               4.9


2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do
restaurante com a menor média de avaliação?

In [38]:
filtro = df1['cuisines'] == 'Italian'

restaurante_italiano_menor_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=True).head(1))
print(restaurante_italiano_menor_media_avaliacao)

              restaurant_name  aggregate_rating
6723  Le Delicatezze Di Bruno               0.0


3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do
restaurante com a maior média de avaliação?

In [39]:
filtro = df1['cuisines'] == 'American'

restaurante_americano_maior_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=False).head(1))
print(restaurante_americano_maior_media_avaliacao)

       restaurant_name  aggregate_rating
726  OEB Breakfast Co.               4.9


4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do
restaurante com a menor média de avaliação?

In [40]:
filtro = df1['cuisines'] == 'American'

restaurante_americano_menor_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=True).head(1))
print(restaurante_americano_menor_media_avaliacao)

        restaurant_name  aggregate_rating
6751  Alston Bar & Beef               0.0


5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do
restaurante com a maior média de avaliação?


In [41]:
filtro = df1['cuisines'] == 'Arabian'

restaurante_arabe_maior_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=False).head(1))
print(restaurante_arabe_maior_media_avaliacao)

     restaurant_name  aggregate_rating
3896        Mandi@36               4.7


6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do
restaurante com a menor média de avaliação?


In [42]:
filtro = df1['cuisines'] == 'Arabian'

restaurante_arabe_menor_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=True).head(1))
print(restaurante_arabe_menor_media_avaliacao)

    restaurant_name  aggregate_rating
337           Raful               0.0


7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do
restaurante com a maior média de avaliação?

In [43]:
filtro = df1['cuisines'] == 'Japanese'

restaurante_japones_maior_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=False).head(1))
print(restaurante_japones_maior_media_avaliacao)

    restaurant_name  aggregate_rating
240    Sushi Leblon               4.9


8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do
restaurante com a menor média de avaliação?


In [44]:
filtro = df1['cuisines'] == 'Japanese'

restaurante_japones_menor_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=True).head(1))
print(restaurante_japones_menor_media_avaliacao)

    restaurant_name  aggregate_rating
123    Banzai Sushi               0.0


9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do
restaurante com a maior média de avaliação?

In [45]:
filtro = df1['cuisines'] == 'Home-made'

restaurante_caseiro_maior_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=False).head(1))
print(restaurante_caseiro_maior_media_avaliacao)

       restaurant_name  aggregate_rating
7525  Kanaat Lokantası               4.0


10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do
restaurante com a menor média de avaliação?

In [46]:
filtro = df1['cuisines'] == 'Home-made'

restaurante_caseiro_menor_media_avaliacao = (df1.loc[filtro, ['restaurant_name', 'aggregate_rating']]
                                              .sort_values(by='aggregate_rating', ascending=True).head(1))
print(restaurante_caseiro_menor_media_avaliacao)

          restaurant_name  aggregate_rating
7373  GurMekan Restaurant               3.7


11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas
pessoas?

In [47]:
culinaria_maior_valor_medio_2_pessoas = (df1.loc[:, ['cuisines', 'average_cost_for_two']]
                                         .groupby('cuisines')
                                         .mean()
                                         .sort_values(by='average_cost_for_two', ascending=False).head(1))
print(culinaria_maior_valor_medio_2_pessoas)

                   average_cost_for_two
cuisines                               
Modern Australian          1.470693e+06


12. Qual o tipo de culinária que possui a maior nota média?

In [48]:
culinaria_maior_nota_media = (df1.loc[:, ['cuisines', 'aggregate_rating']]
                              .sort_values(by='aggregate_rating', ascending=False).head(1))
print(culinaria_maior_nota_media)

     cuisines  aggregate_rating
1535  Italian               4.9


13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos
online e fazem entregas?

In [49]:
filtro = (df1['has_table_booking'] == 1) & (df1['has_online_delivery'] == 1)

culinaria_mais_reservas_e_pedidos_online = (df1.loc[filtro, ['cuisines', 'restaurant_name']]
                                            .groupby('cuisines')
                                            .nunique()
                                            .sort_values(by='restaurant_name', ascending=False).head(1))
print(culinaria_mais_reservas_e_pedidos_online)

              restaurant_name
cuisines                     
North Indian               33
